In [161]:
import os
import pandas as pd
import re

Now that we are in the preferable working directory, I'll explore each data file's variables in order to figure out how to join each dataframe into a big dataset. Let's begin chronologically:

# 2000-2020

In [5]:
df_2000=pd.read_csv(f"{data_path}/2000.csv")
df_2005=pd.read_csv(f"{data_path}/2005.csv")
df_2010=pd.read_csv(f"{data_path}/2010.csv")
df_2020=pd.read_csv(f"{data_path}/2020.csv")

/var/folders/m5/26l5nfs54r94_664jm4k6zvr0000gn/T/ipykernel_9866/36002156.py:4: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2020=pd.read_csv(f"{data_path}/2020.csv")


I think it will be a good idea to split the population columns into categories. For example, the 'Población de 15 años y más' column will be abreviated as 'POB_15+', and each description will be abreviated as well.

I'll begin with abreviating the first part of the population columns.









Now, I need to find a good use of regular expressions in order to speed things up. I want for the next columns to have the name structure: "POB_[age_range]_[category]". 

In [192]:

def population(column_name):
    match=re.search("Población",column_name)
    if match:
        sex=re.search("femenina|masculina",column_name)
        if sex:
            letter=str.upper(sex.group()[0])
            return f"POB {letter}"
        return "POB"
    
def age_range(column_name):
    """ 
    
    Takes a column name as an input and spits out the age range the input contains
    
    """
    # Case for specified age range:
    age_range_pattern=r'\d+ a \d+' 
    match=re.search(age_range_pattern,column_name)
    if match:
        age_range=match.group()
        #age_range=str.replace(age_range," ","_")
        return age_range
    
    # Case for ambiguos age range:
    ambiguos_age_range_pattern=r"\d+ años y más"
    match=re.search(ambiguos_age_range_pattern,column_name)
    if match:
        text=match.group(0)
        age_range=f"{re.findall(r'\d+',column_name)[0]}+"
        return age_range
    
def category(column_name):
    """ 
    
    Takes a column name as an input and spits out the category wanted depending on what the input contains
    
    """

    categories={
         #Education
        "que no asiste a la escuela":"NAE",
        "que asiste a la escuela":"AE",
        "que sabe leer y escribir":"SLE",
        "que no sabe leer y escribir":"NSLE",
        "analfabeta":"ANBT",
        "sin escolaridad":"SIN_E",
        "primaria completa":"PRIM_COM",
        "primaria incompleta":"PRIM_INC",
        "secundaria completa":"SEC_COM",
        "secundaria incompleta":"SEC_INC",
        "educación posbásica":"POSBASICA",

        #Financial
        "económicamente activa":"EC_ACTIVA",
        "no económicamente activa":"NO_EC_ACTIVA",
        "ocupada":"EMPLEADA",
        "desocupada":"DESEMPLEADA"


    }
    for replacement,category in categories.items():
        match=re.search(fr'{replacement}',column_name)
        if match:
            return category
    #print("No matches :( ")


def grade(column_name):
        match=re.search("Grado promedio",column_name)
        if match:
            sex=re.search("femenina|masculina",column_name)
            if sex:
                letter=str.upper(sex.group()[0])
                return f"GDO {letter}"
            return "GDO"
        

def other(column_name):
        others={
        #2020
         'Clave de entidad federativa':'CVE_ENT',
         'Entidad federativa':'ENT',
         'Clave de municipio o demarcación territorial':'CVE TERR',
         'Municipio o demarcación territorial':'MUN',
         'Clave de localidad':'CVE_LOC',
         'Localidad':'LOCALIDAD',
         'Longitud':'LON',
         'Latitud':'LAT',
         'Altitud':'ALTI',
         'Población total':'POB_TOT',
         
        #2010
        'Clave de municipio ó delegación':'CVE TERR',
        'Municipio ó delegación':'MUN',

        #2005
        'Entidad Federativa':'ENT',
        'Clave del Municipio':'CVE TERR',
        'Municipio':'MUN',

        #2000
        'Nombre de la entidad':'ENT',
        'Clave de municipio o delegación':'CVE TERR',
        'Nombre del municipio o delegación':'MUN',
        'Nombre de la localidad':'LOCALIDAD',
        'Po blación femenina de 15 a 49 años':'POB_15_a_49'
          }
        


        for replacement,other in others.items():
            match=re.search(fr'{replacement}',column_name)
            if match:
                return other
            

def rename_columns(df):
        original_columns={}
    
        for column in df.columns:

            # Other columns:
            if other(column):
                original_columns[column]=other(column)
                continue

            # Population columns
            if population(column):
                if age_range(column):
                    if category(column):
                        #print(column, f"{population(column)}_{age_range(column)}_{category(column)}")
                        original_columns[column]=f"{population(column)}_{age_range(column)}_{category(column)}"
                        continue
            
                    #print(column, f"{population(column)}_{age_range(column)}")
                    original_columns[column]=f"{population(column)}_{age_range(column)}"
                    continue

                #print(column, f"{population(column)}")
                original_columns[column]=f"{population(column)}"
                continue

            # Grade Columns
            if grade(column):
                original_columns[column]=f"{grade(column)}"
                #print(column,f"{grade(column)}")
                continue
            
        
        df=df.rename(columns=original_columns)
            
        return [df,original_columns]

In [193]:
rename_columns(df_2020)[0].head(3)

,Unnamed: 0,CVE_ENT,ENT,CVE TERR,MUN,CVE_LOC,LOCALIDAD,LON,LAT,ALTI,...,POB M_12+_EC_ACTIVA,POB_12+_EC_ACTIVA,POB F_12+_EC_ACTIVA,POB M_12+_EC_ACTIVA,POB_12+_EMPLEADA,POB F_12+_EMPLEADA,POB M_12+_EMPLEADA,POB_12+_EMPLEADA,POB F_12+_EMPLEADA,POB M_12+_EMPLEADA
0,0,0,Total nacional,0,Total nacional,0,Total nacional,NaN,NaN,NaN,...,36815941,37891261,26379060,11512201,61121324,25137019,35984305,1160310,328674,831636
1,1,0,Total nacional,0,Total nacional,9998,Localidades de una vivienda,NaN,NaN,NaN,...,95222,68766,42827,25939,127469,32906,94563,820,161,659
2,2,0,Total nacional,0,Total nacional,9999,Localidades de dos viviendas,NaN,NaN,NaN,...,50429,46162,28959,17203,68215,18365,49850,689,110,579


In [ ]:
rename_columns(df_2020)[0]

In [194]:
rename_columns(df_2010)[0].head(3)

,Unnamed: 0,CVE_ENT,ENT,CVE TERR,MUN,CVE_LOC,LOCALIDAD,LON,LAT,ALTI,...,POB F_15+_SEC_INC,POB_15+_SEC_COM,POB M_15+_SEC_COM,POB F_15+_SEC_COM,POB_18+,POB M_18+,POB F_18+,GDO,GDO M,GDO F
0,0,0,Total nacional,0,Total nacional,0,Total nacional,NaN,NaN,NaN,...,1848029,17181221,8491006,8690215,26057800,12797492,13260308,8.63,8.79,8.48
1,3,1,Aguascalientes,0,Total de la entidad Aguascalientes,0,Total de la Entidad,NaN,NaN,NaN,...,17250,203516,97269,106247,283254,136375,146879,9.23,9.30,9.17
2,6,1,Aguascalientes,1,Aguascalientes,0,Total del Municipio,NaN,NaN,NaN,...,11281,133433,64820,68613,226503,109237,117266,9.81,9.95,9.69


In [195]:
rename_columns(df_2005)[0].head(3)

,Unnamed: 0,ENT,CVE TERR,MUN,LOCALIDAD,LOCALIDAD,LON,LAT,ALTI,POB_TOT,...,POB F_15+,POB_15+,POB M_15+,POB F_15+,POB_15+_POSBASICA,POB M_15+_POSBASICA,POB F_15+_POSBASICA,GDO,GDO M,GDO F
0,0,Total nacional,0,Total nacional,0,Total nacional,NaN,NaN,0.0,103263388,...,13418938,14533348,7179086,7354262,22063108,10900960,11162148,8.14,8.37,7.93
1,1,Total nacional,0,Total nacional,9998,Localidades de una vivienda,NaN,NaN,0.0,275182,...,40959,25287,15889,9398,19344,12098,7246,4.98,5.11,4.79
2,2,Total nacional,0,Total nacional,9999,Localidades de dos viviendas,NaN,NaN,0.0,179318,...,28161,15411,8980,6431,11006,6639,4367,4.81,4.98,4.60


In [196]:
rename_columns(df_2000)[0].head(3)

,Unnamed: 0,CVE_ENT,ENT,CVE TERR,MUN,CVE_LOC,LOCALIDAD,LON,LAT,ALTI,...,POB_15+,POB_15+_SEC_INC,POB_15+_SEC_COM,POB_15+,POB_15+,POB_15+,POB_18+,POB_18+,POB_18+,GDO
0,0,0,Total nacional,0,Total nacional,0,TOTAL NACIONAL,NaN,NaN,NaN,...,32535609,3205083,11603530,29757418,15210653,17324956,40335005,8999238,6861605,7.0
1,1,0,Total nacional,0,Total nacional,9999,LOCALIDADES DE DOS VIVIENDAS,NaN,NaN,NaN,...,24768,5174,11524,102330,16827,7941,106671,4733,2361,NaN
2,2,0,Total nacional,0,Total nacional,9998,LOCALIDADES DE UNA VIVIENDA,NaN,NaN,NaN,...,45244,10013,20266,159126,30531,14713,171116,9085,4256,NaN


In [203]:
rename_columns(df_2000)[0].columns

Index(['Unnamed: 0', 'CVE_ENT', 'ENT', 'CVE TERR', 'MUN', 'CVE_LOC',
       'LOCALIDAD', 'LON', 'LAT', 'ALTI', 'POB_TOT', 'POB M', 'POB F',
       'POB_0 a 4', 'POB_5+', 'POB_6 a 14', 'POB_12+', 'POB_15+',
       'POB_15 a 17', 'POB_15 a 24', 'POB_15_a_49', 'POB_18+', 'POB M_18+',
       'POB F_18+', 'POB_6 a 14', 'POB_6 a 14_NSLE', 'POB_15+', 'POB_15+_ANBT',
       'POB', 'POB', 'POB_6 a 14_AE', 'POB_6 a 14_NAE', 'POB_15 a 17_AE',
       'POB_15 a 24_AE', 'POB_15 a 24_NAE', 'POB_15+', 'POB_15+_PRIM_INC',
       'POB_15+_PRIM_COM', 'POB_15+', 'POB_15+_SEC_INC', 'POB_15+_SEC_COM',
       'POB_15+', 'POB_15+', 'POB_15+', 'POB_18+', 'POB_18+', 'POB_18+',
       'GDO'],
      dtype='object')

In [204]:
rename_columns(df_2000)[1]

{'Clave de entidad federativa': 'CVE_ENT',
 'Nombre de la entidad': 'ENT',
 'Clave de municipio o delegación': 'CVE TERR',
 'Nombre del municipio o delegación': 'MUN',
 'Clave de localidad': 'CVE_LOC',
 'Nombre de la localidad': 'LOCALIDAD',
 'Longitud': 'LON',
 'Latitud': 'LAT',
 'Altitud': 'ALTI',
 'Población total': 'POB_TOT',
 'Población masculina': 'POB M',
 'Población femenina': 'POB F',
 'Población de 0 a 4 años': 'POB_0 a 4',
 'Población de 5 años y más': 'POB_5+',
 'Población de 6 a 14 años': 'POB_6 a 14',
 'Población de 12 años y más': 'POB_12+',
 'Población de 15 años y más': 'POB_15+',
 'Población de 15 a 17 años': 'POB_15 a 17',
 'Población de 15 a 24 años': 'POB_15 a 24',
 'Po blación femenina de 15 a 49 años': 'POB_15_a_49',
 'Población de 18 años y más': 'POB_18+',
 'Población masculina de 18 años y más': 'POB M_18+',
 'Población femenina de 18 años y más': 'POB F_18+',
 'Población de 6 a 14 años que sabe leer y escribir': 'POB_6 a 14',
 'Población de 6 a 14 años que no

Now that I have a good function to rename my variables, all we need now is to filter the columns we aren't interested in,(again): 

In [180]:
def filter(df):
    
    columns_staying=[]
    columns_to_filter_out=[
        # Disabilities
        'Población con discapacidad', 
        'Población con discapacidad para hablar o comunicarse', 
        'Población con discapacidad para oír, aun usando aparato auditivo', 
        'Población con discapacidad para vestirse, bañarse o comer', 
        'Población con discapacidad para recordar o concentrarse', 
        'Población con limitación', 
        'Población con limitación para caminar, subir o bajar', 
        'Población con limitación para ver, aun usando lentes', 
        'Población con limitación para hablar o comunicarse', 
        'Población con limitación para oír, aun usando aparato auditivo', 
        'Población con limitación para vestirse, bañarse o comer', 
        'Población con limitación para recordar o concentrarse', 
        'Población con algún problema o condición mental', 
        'Población sin discapacidad, limitación, problema o condición mental',
        # Financial
        
       'Población de 12 años y más económicamente activa',
       'Población femenina de 12 años y más económicamente activa',
       'Población masculina de 12 años y más económicamente activa',
       'Población de 12 años y más no económicamente activa',
       'Población femenina de 12 años y más no económicamente activa',
       'Población masculina de 12 años y más no económicamente activa',
       'Población de 12 años y más ocupada',
       'Población femenina de 12 años y más ocupada',
       'Población masculina de 12 años y más ocupada',
       'Población de 12 años y más desocupada',
       'Población femenina de 12 años y más desocupada',
       'Población masculina de 12 años y más desocupada'
       ]
    
    for column in df.columns:

        if column in columns_to_filter_out:
            continue

        columns_staying.append(column)

    filtered_df=df[columns_staying]
    
    return filtered_df

In [202]:
filter(df_2020).columns

Index(['Unnamed: 0', 'Clave de entidad federativa', 'Entidad federativa',
       'Clave de municipio o demarcación territorial',
       'Municipio o demarcación territorial', 'Clave de localidad',
       'Localidad', 'Longitud', 'Latitud', 'Altitud',
       ...
       'Población masculina de 15 años y más con secundaria incompleta',
       'Población de 15 años y más con secundaria completa',
       'Población femenina de 15 años y más con secundaria completa',
       'Población masculina de 15 años y más con secundaria completa',
       'Población de 18 años y más con educación posbásica',
       'Población femenina de 18 años y más con educación posbásica',
       'Población masculina de 18 años y más con educación posbásica',
       'Grado promedio de escolaridad',
       'Grado promedio de escolaridad de la población femenina',
       'Grado promedio de escolaridad de la población masculina'],
      dtype='object', length=145)

In [201]:
filter(df_2010).columns

Index(['Unnamed: 0', 'Clave de entidad federativa', 'Entidad federativa',
       'Clave de municipio ó delegación', 'Municipio ó delegación',
       'Clave de localidad', 'Localidad', 'Longitud', 'Latitud', 'Altitud',
       ...
       'Población femenina de 15 años y más con secundaria incompleta',
       'Población de 15 años y más con secundaria completa',
       'Población masculina de 15 años y más con secundaria completa',
       'Población femenina de 15 años y más con secundaria completa',
       'Población de 18 años y más con educación pos-básica',
       'Población masculina de 18 años y más con educación pos-básica',
       'Población femenina de 18 años y más con educación pos-básica',
       'Grado promedio de escolaridad',
       'Grado promedio de escolaridad de la población masculina',
       'Grado promedio de escolaridad de la población femenina'],
      dtype='object', length=106)

In [200]:
filter(df_2005).columns

Index(['Unnamed: 0', 'Entidad Federativa', 'Clave del Municipio', 'Municipio',
       'Clave de Localidad', 'Localidad', 'Longitud', 'Latitud', 'Altitud',
       'Población total', 'Población masculina', 'Población femenina',
       'Población de 8 a 14 años que no sabe leer y escribir',
       'Población masculina de 8 a 14 años que no sabe leer y escribir',
       'Población femenina de 8 a 14 años que no sabe leer y escribir',
       'Población de 15 años y más analfabeta',
       'Población masculina de 15 años y más analfabeta',
       'Población femenina de 15 años y más analfabeta',
       'Población de 5 años que no asiste a la escuela',
       'Población masculina de 5 años que no asiste a la escuela',
       'Población femenina de 5 años que no asiste a la escuela',
       'Población de 6 a 11 años que no asiste a la escuela',
       'Población de 6 a 14 años que no asiste a la escuela',
       'Población masculina de 6 a 14 años que no asiste a la escuela',
       'Población

In [186]:
filter(df_2000).columns

Index(['Unnamed: 0', 'Clave de entidad federativa', 'Nombre de la entidad',
       'Clave de municipio o delegación', 'Nombre del municipio o delegación',
       'Clave de localidad', 'Nombre de la localidad', 'Longitud', 'Latitud',
       'Altitud', 'Población total', 'Población masculina',
       'Población femenina', 'Población de 0 a 4 años',
       'Población de 5 años y más', 'Población de 6 a 14 años',
       'Población de 12 años y más', 'Población de 15 años y más',
       'Población de 15 a 17 años', 'Población de 15 a 24 años',
       'Po blación femenina de 15 a 49 años', 'Población de 18 años y más',
       'Población masculina de 18 años y más',
       'Población femenina de 18 años y más',
       'Población de 6 a 14 años que sabe leer y escribir',
       'Población de 6 a 14 años que no sabe leer y escribir',
       'Población de 15 años y más alfabeta',
       'Población de 15 años y más analfabeta',
       'Población de 5 años que asiste a la escuela',
       'Poblaci

Now comes the hard part: concatenating these bad boys.

# Tests

In [122]:
for column in df_2020:
    if column not in rename_columns(df_2020):
        print(column)

Unnamed: 0
Relación hombres-mujeres


The 'Relación hombres-mujeres' column is not necessary and the values in it can be obtained from the 'Población masculina' and 'Población femenina' columns. 

In [127]:
for column in df_2010:
    if column not in rename_columns(df_2010):
        print(column)

Unnamed: 0
Relación hombres-mujeres


In [135]:
for column in df_2005:
    if column not in rename_columns(df_2005):
        print(column)

Unnamed: 0


In [137]:
for column in df_2000:
    if column not in rename_columns(df_2000):
        print(column)

Unnamed: 0
